In [ ]:
%pip install langchain-community

In [ ]:
import sys
import logging
from pathlib import Path

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

PROJECT_ID = "dark-data-discovery"
LOCATION = "us-central1"

print(f"📍 GCP Project: {PROJECT_ID}")
print(f"📍 Location: {LOCATION}")

In [ ]:
from google.cloud import secretmanager

def get_secret(project_id, secret_id, version_id="latest"):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

try:
    key = get_secret(PROJECT_ID, "ANTHROPIC_API_KEY")
    print(f"✅ ANTHROPIC_API_KEY: {'*' * 10}{key[-4:]}")
except Exception as e:
    print(f"❌ Failed to get ANTHROPIC_API_KEY: {e}")

In [ ]:
from hegemon import create_hegemon_graph, get_settings, DebateState, FinalPlan

print("✅ HEGEMON imported!")
print(f"Version: {__import__('hegemon').__version__}")

In [ ]:
settings = get_settings()

print(f"✅ Settings initialized")
print(f"\nConfiguration:")
print(f"  Project: {settings.gcp_project_id}")
print(f"  Location: {settings.gcp_location}")
print(f"  Katalizator: {settings.katalizator.model} ({settings.katalizator.provider})")
print(f"  Sceptyk: {settings.sceptyk.model} ({settings.sceptyk.provider}, Vertex AI: {settings.sceptyk.use_vertex_ai})")
print(f"  Gubernator: {settings.gubernator.model} ({settings.gubernator.provider})")
print(f"  Syntezator: {settings.syntezator.model} ({settings.syntezator.provider})")
print(f"\nDebate Config:")
print(f"  Consensus threshold: {settings.debate.consensus_threshold}")
print(f"  Max cycles: {settings.debate.max_cycles}")

In [ ]:
graph = create_hegemon_graph()

print("✅ HEGEMON graph created!")
print(f"\nGraph structure:")
print(f"  Nodes: {list(graph.nodes.keys())}")

In [ ]:
MISSION = """
Design automated book layout pipeline using InDesign scripting for 280-page 
technical programming book, executable by AI agents with access to Adobe InDesign API, 
Python scripting, and document processing tools.

BOOK SPECIFICATIONS:
- Title: "Advanced Python Patterns: A Practitioner's Guide"
- Content: 12 chapters, 280 pages (pre-formatted manuscript in Markdown/LaTeX)
- Format: 7" x 10" trade paperback (Crown Quarto)
- Output: Print-ready PDF (IngramSpark specs) + ePub
- Audience: professional software engineers

CONTENT COMPLEXITY:
- 50 code blocks (10-200 lines each, Python syntax highlighting)
- 30 technical diagrams (provided as SVG/PNG, 300 DPI)
- Mathematical notation (LaTeX format in source)
- 3 callout box types with structured markup
- Index: 500+ entries (extractable from source annotations)
- Bibliography: 120 citations (BibTeX format provided)

EXECUTION CONSTRAINTS (AI-OPTIMIZED):
- Timeline: Complete pipeline execution in <24 hours wall-clock time
- Compute budget: $50 total (API calls, cloud compute)
- Tools available to agents:
  * InDesign Server API (scripting via ExtendScript/JavaScript)
  * Python 3.11+ with libraries: reportlab, PyPDF2, lxml, beautifulsoup4
  * Pandoc for format conversion (Markdown → ICML)
  * Pygments for syntax highlighting (pre-processing step)
  * Adobe Preflight API for PDF/X validation
  * PAC 2024 for WCAG accessibility checking
- Manuscript format: Structured Markdown with frontmatter (YAML metadata)
- No human-in-loop approvals (fully automated validation)

AUTOMATION REQUIREMENTS:
- Template generation: Programmatic master page creation via InDesign scripting
- Style application: Paragraph/character styles applied via GREP and scripting
- Code highlighting: Pre-process with Pygments, import as styled text
- Figure placement: Automated anchoring based on markup (e.g., `![Figure 3.2](path/to/diagram.svg)`)
- Index generation: Extract from `\index{term}` tags in source, auto-generate with InDesign API
- Cross-references: Automated figure/table numbering via InDesign cross-ref system
- Accessibility tagging: Automated structure tags based on semantic HTML export from Pandoc

DELIVERABLES (ALL PROGRAMMATICALLY GENERATED):
- InDesign template (.indt) created via scripting
- Print-ready PDF/X-1a passing Adobe Preflight (0 errors)
- Tagged PDF passing PAC accessibility check (WCAG 2.1 AA)
- ePub 3.0 with reflowable layout
- Automated validation report (JSON format with all quality metrics)
- Reproducible pipeline (Docker container or shell script)

SUCCESS METRICS (AUTOMATED VALIDATION):
- IngramSpark preflight: PASS with 0 critical errors
- Accessibility check: PASS with 0 WCAG 2.1 AA violations
- Code font legibility: 9pt minimum (validated via PDF text extraction)
- Index accuracy: Auto-generated, cross-checked against source tags
- Build reproducibility: Pipeline runs identically on re-execution
- Processing time: <4 hours for full 280-page layout
- API cost: <$50 (primarily InDesign Server API calls + LLM tool use)

TECHNICAL SPECIFICATIONS:
- Print: 300 DPI, CMYK (Coated FOGRA39), 0.125" bleed, PDF/X-1a:2001
- Typography: 
  * Body: Minion Pro 10.75pt (embedded subset)
  * Headings: Myriad Pro (4 levels: 24pt, 18pt, 14pt, 12pt)
  * Code: Source Code Pro 9pt (monospace)
- Color: Black + Pantone 2738 C (accent - converted to CMYK for print)
- Page geometry: 7" × 10", margins (inner: 0.75", outer: 1.5", top: 0.75", bottom: 1")

AGENT CAPABILITIES ASSUMED:
- Code execution: Python, JavaScript (ExtendScript)
- File operations: Read/write local filesystem, download from URLs
- API calls: REST APIs (InDesign Server, Preflight services)
- Document parsing: Markdown, LaTeX, YAML, XML (ICML), JSON
- Text processing: Regex, GREP patterns, string manipulation
- Validation: PDF analysis, accessibility tree inspection

FAILURE MODES TO HANDLE:
- Invalid source markup → Validation error with line numbers
- Missing diagrams → Placeholder with error message in output
- Font embedding issues → Fallback to safe font list
- CMYK conversion problems → Use sRGB with warning flag
- Preflight failures → Return detailed error report for manual review

OUTPUT FORMAT:
Execution plan must specify:
1. Agent roles with specific tools assigned
2. Workflow as DAG with parallelizable steps
3. Automated validation checkpoints (not human approval)
4. Error handling and retry logic
5. Cost estimation per step (API calls, compute time)
6. Final deliverable verification (checksums, validation reports)
"""

print("📋 Mission defined:")
print(MISSION)

In [ ]:
initial_state = {
    "mission": MISSION,
    "contributions": [],
    "cycle_count": 1,
    "current_consensus_score": 0.0,
    "final_plan": None,
}

print("🚀 Starting HEGEMON debate...\n")
print("=" * 80)

final_state = graph.invoke(initial_state)

print("\n" + "=" * 80)
print("✅ Debate complete!")

In [ ]:
print("\n" + "=" * 80)
print("📊 DEBATE SUMMARY")
print("=" * 80 + "\n")

print(f"Total cycles: {final_state['cycle_count']}")
print(f"Final consensus score: {final_state['current_consensus_score']:.2f}")
print(f"Total contributions: {len(final_state['contributions'])}")

print("\n" + "=" * 80)
print("📝 DEBATE HISTORY")
print("=" * 80 + "\n")

for contrib in final_state['contributions']:
    print(f"\n{'='*80}")
    print(f"Agent: {contrib.agent_id} | Type: {contrib.type} | Cycle: {contrib.cycle}")
    print(f"{'='*80}")
    print(f"\n{contrib.content[:500]}...")
    print(f"\nRationale: {contrib.rationale}")

In [ ]:
final_plan = final_state['final_plan']

print("\n" + "=" * 80)
print("🎯 FINAL STRATEGIC PLAN")
print("=" * 80 + "\n")

print("📋 MISSION OVERVIEW:")
print("-" * 80)
print(final_plan.mission_overview)

print("\n\n👥 REQUIRED AGENTS:")
print("-" * 80)
for agent in final_plan.required_agents:
    print(f"\n{agent.role}")
    print(f"  Description: {agent.description}")
    print(f"  Skills: {', '.join(agent.required_skills)}")

print("\n\n📊 WORKFLOW ({} steps):".format(len(final_plan.workflow)))
print("-" * 80)
for step in final_plan.workflow:
    print(f"\nStep {step.step_id}: {step.assigned_agent_role}")
    print(f"  Task: {step.description}")
    if step.dependencies:
        print(f"  Dependencies: {step.dependencies}")

print("\n\n⚠️ RISK ANALYSIS:")
print("-" * 80)
print(final_plan.risk_analysis)

In [ ]:
# ============================================================================
# CELL 13: Export to JSON
# ============================================================================
import json
from datetime import datetime

output = {
    "timestamp": datetime.now().isoformat(),
    "mission": final_state['mission'],
    "debate_summary": {
        "total_cycles": final_state['cycle_count'],
        "final_consensus_score": final_state['current_consensus_score'],
        "total_contributions": len(final_state['contributions']),
    },
    "debate_history": [
        {
            "agent_id": c.agent_id,
            "type": c.type,
            "cycle": c.cycle,
            "content": c.content,
            "rationale": c.rationale,
        }
        for c in final_state['contributions']
    ],
    "final_plan": {
        "mission_overview": final_plan.mission_overview,
        "required_agents": [
            {
                "role": a.role,
                "description": a.description,
                "required_skills": a.required_skills,
            }
            for a in final_plan.required_agents
        ],
        "workflow": [
            {
                "step_id": s.step_id,
                "description": s.description,
                "assigned_agent_role": s.assigned_agent_role,
                "dependencies": s.dependencies,
            }
            for s in final_plan.workflow
        ],
        "risk_analysis": final_plan.risk_analysis,
    }
}

output_path = Path("output")
output_path.mkdir(exist_ok=True)

filename = output_path / f"hegemon_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

with open(filename, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

print(f"✅ Results exported to: {filename}")
print(f"📊 File size: {filename.stat().st_size / 1024:.1f} KB")

In [ ]:
# ============================================================================
# CELL 14: Visualize Workflow as Graph (Optional)
# ============================================================================
try:
    import matplotlib.pyplot as plt
    import networkx as nx
    
    G = nx.DiGraph()
    
    for step in final_plan.workflow:
        G.add_node(step.step_id, label=f"Step {step.step_id}\n{step.assigned_agent_role}")
        for dep in step.dependencies:
            G.add_edge(dep, step.step_id)
    
    plt.figure(figsize=(14, 10))
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    nx.draw(G, pos, 
            node_color='lightblue', 
            node_size=3000,
            with_labels=True,
            labels={n: f"Step {n}" for n in G.nodes()},
            font_size=10,
            font_weight='bold',
            arrows=True,
            arrowsize=20,
            edge_color='gray',
            width=2)
    
    plt.title("Workflow Dependency Graph", fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_path / "workflow_graph.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Workflow graph saved!")
    
except ImportError:
    print("⚠️ matplotlib/networkx not installed. Skipping visualization.")
    print("Install with: pip install matplotlib networkx")

In [ ]:
# ============================================================================
# CELL 15: Quick Stats
# ============================================================================
print("\n" + "=" * 80)
print("📈 QUICK STATS")
print("=" * 80 + "\n")

print(f"Mission length: {len(MISSION)} characters")
print(f"Debate cycles: {final_state['cycle_count']}")
print(f"Final consensus: {final_state['current_consensus_score']:.2%}")
print(f"Total contributions: {len(final_state['contributions'])}")
print(f"  - Theses: {sum(1 for c in final_state['contributions'] if c.type == 'Thesis')}")
print(f"  - Antitheses: {sum(1 for c in final_state['contributions'] if c.type == 'Antithesis')}")
print(f"  - Evaluations: {sum(1 for c in final_state['contributions'] if c.type == 'Evaluation')}")
print(f"  - Final Plans: {sum(1 for c in final_state['contributions'] if c.type == 'FinalPlan')}")

print(f"\nFinal Plan:")
print(f"  - Required agents: {len(final_plan.required_agents)}")
print(f"  - Workflow steps: {len(final_plan.workflow)}")
print(f"  - Mission overview length: {len(final_plan.mission_overview)} chars")
print(f"  - Risk analysis length: {len(final_plan.risk_analysis)} chars")

total_chars = sum(len(c.content) for c in final_state['contributions'])
print(f"\nTotal debate output: {total_chars:,} characters ({total_chars/1024:.1f} KB)")

In [1]:
# Komórka 1: Setup
import os
import sys
import json
import logging
from pathlib import Path

# ENABLE EXPLAINABILITY (przez os.environ)
os.environ["HEGEMON_EXPLAINABILITY_ENABLED"] = "true"
os.environ["HEGEMON_EXPLAINABILITY_SEMANTIC_FINGERPRINT"] = "true"
os.environ["HEGEMON_EXPLAINABILITY_CLASSIFIER_MODEL"] = "gemini-2.5-flash"  # Vertex AI model

# Setup logging dla Jupyter
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True
)
logger = logging.getLogger(__name__)

print("✅ Environment configured for Vertex AI")
print(f"   Explainability enabled: {os.environ.get('HEGEMON_EXPLAINABILITY_ENABLED')}")
print(f"   Classifier model: {os.environ.get('HEGEMON_EXPLAINABILITY_CLASSIFIER_MODEL')}")

✅ Environment configured for Vertex AI
   Explainability enabled: true
   Classifier model: gemini-2.5-flash


In [2]:
# Komórka 2: Import Hegemon modules
from hegemon.graph import create_hegemon_graph
from hegemon.explainability.visualizer import HeatmapGenerator
from hegemon.config.settings import get_settings

# Verify settings
settings = get_settings()
print("Explainability Layers:")
print(f"  Layer 6 (Semantic): {settings.explainability_semantic_fingerprint}")
print(f"  Layer 2 (Epistemic): {settings.explainability_epistemic_uncertainty}")


print("=" * 80)
print("✅ STEP 2: Settings loaded and verified")
print("=" * 80)
print(f"GCP Project ID: {settings.gcp_project_id}")
print(f"GCP Location: {settings.gcp_location}")
print()
print("🔍 EXPLAINABILITY SETTINGS:")
print(f"   enabled: {settings.explainability_enabled}")
print(f"   semantic_fingerprint: {settings.explainability_semantic_fingerprint}")
print(f"   classifier_model: {settings.explainability_classifier_model}")
print(f"   cache_size: {settings.explainability_cache_size}")
print("=" * 80)

# CRITICAL CHECK
if not settings.explainability_enabled:
    print("\n❌ ERROR: Explainability is DISABLED!")
    print("   This means environment variable was set AFTER settings were cached.")
    print("   SOLUTION: Restart kernel and run cells in correct order.")
else:
    print("\n✅ SUCCESS: Explainability is ENABLED!")
    print("   Proceed to next cells.")

print()

2025-10-03 13:51:24,940 - hegemon.config.settings - INFO - ✅ ANTHROPIC_API_KEY loaded from Secret Manager
2025-10-03 13:51:24,941 - hegemon.config.settings - INFO - ✅ Google Gemini will use Vertex AI (ADC authentication)
2025-10-03 13:51:25,000 - hegemon.config.settings - INFO - ✅ OPENAI_API_KEY loaded from Secret Manager
/opt/conda/envs/hegemon_env/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py:63: UserWarning: `validate_explainability_config` overrides an existing Pydantic `@model_validator` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')
2025-10-03 13:51:28,751 - hegemon.config.settings - WARNING - Explainability enabled but no Google API key found. Classifier will fail.
2025-10-03 13:51:28,752 - hegemon.config.settings - INFO - ✅ Sceptyk will use Vertex AI (Application Default Credentials)
2025-10-03 13:51:28,752 - hegemon.config.settings - INFO - ✅ All required authentication validated


Explainability Layers:
  Layer 6 (Semantic): True
  Layer 2 (Epistemic): True
✅ STEP 2: Settings loaded and verified
GCP Project ID: dark-data-discovery
GCP Location: us-central1

🔍 EXPLAINABILITY SETTINGS:
   enabled: True
   semantic_fingerprint: True
   classifier_model: gemini-2.0-flash
   cache_size: 1000

✅ SUCCESS: Explainability is ENABLED!
   Proceed to next cells.



In [3]:
# Komórka 3: Define mission
mission = """
Design a comprehensive AI-powered customer service automation strategy 
for a mid-sized e-commerce company (500 employees, $50M annual revenue).

Requirements:
- Reduce customer service costs by 30%
- Maintain or improve customer satisfaction scores
- Implementation timeline: 6 months
- Budget: $500,000
- Must integrate with existing Salesforce CRM

Deliverable: Detailed implementation plan with agent roles, workflow, 
and risk analysis.
"""

# initial_state = {
#     "mission": mission,
#     "contributions": [],
#     "cycle_count": 1,
#     "current_consensus_score": 0.0,
#     "final_plan": None,
# }


# Użyj reviewer mode
initial_state = {
    "mission": "Zaprojektuj ML pipeline",
    "contributions": [],
    "cycle_count": 1,
    "current_consensus_score": 0.0,
    "final_plan": None,
    "intervention_mode": "observer",  # Włącz checkpoints
    "current_checkpoint": None,
    "human_feedback_history": [],
    "paused_at": None,
    "revision_count_per_checkpoint": {},
    "checkpoint_snapshots": {}
}


print("✅ Mission defined")
print("=" * 80)
print(mission)
print("=" * 80)

✅ Mission defined

Design a comprehensive AI-powered customer service automation strategy 
for a mid-sized e-commerce company (500 employees, $50M annual revenue).

Requirements:
- Reduce customer service costs by 30%
- Maintain or improve customer satisfaction scores
- Implementation timeline: 6 months
- Budget: $500,000
- Must integrate with existing Salesforce CRM

Deliverable: Detailed implementation plan with agent roles, workflow, 
and risk analysis.



In [4]:
from hegemon.graph_hitl import create_hegemon_graph_hitl
from hegemon.hitl import HumanFeedback

In [5]:
# Komórka 4: Create Hegemon graph
graph = create_hegemon_graph_hitl()

print("✅ Hegemon graph created")
print(f"   Nodes: {list(graph.nodes.keys())}")

2025-10-03 13:51:28,790 - hegemon.graph_hitl - INFO - 🏗️ Building HEGEMON graph with HITL checkpoints...
2025-10-03 13:51:28,812 - hegemon.graph_hitl - INFO - ✅ HEGEMON graph with HITL compiled successfully!
2025-10-03 13:51:28,813 - hegemon.graph_hitl - INFO -    Checkpoints: post_thesis, post_evaluation, pre_synthesis


✅ Hegemon graph created
   Nodes: ['__start__', 'katalizator', 'sceptyk', 'gubernator', 'syntezator', 'increment_cycle', 'checkpoint_post_thesis', 'checkpoint_post_evaluation', 'checkpoint_pre_synthesis']


In [6]:
# Komórka 5: Run debate
print("🎭 Starting dialectical debate...")
print("⏳ This will take 2-5 minutes...\n")

final_state = graph.invoke(initial_state,config={"recursion_limit": 100} )


# Auto-export Layer 2 data
from hegemon.explainability.visualizer import (
    export_all_epistemic_profiles,
    create_epistemic_comparison_report,
)

print("\n" + "=" * 80)
print("EXPORTING LAYER 2 DATA...")
print("=" * 80)

exported = export_all_epistemic_profiles(
    final_state["contributions"],
    output_dir="epistemic_exports",
    format="both"
)

report = create_epistemic_comparison_report(
    final_state["contributions"],
    output_path="epistemic_comparison.md"
)


def export_hitl_feedback_summary(state: dict, output_path: str = "hitl_summary.md") -> str:
    """
    Generate markdown report of human interventions.
    
    Shows:
    - Which checkpoints were reached
    - What decisions were made (approve/revise/reject)
    - What guidance was provided
    - How many revisions per checkpoint
    """
    from pathlib import Path
    
    feedback_history = state.get("human_feedback_history", [])
    revision_counts = state.get("revision_count_per_checkpoint", {})
    checkpoints_reached = state.get("checkpoint_snapshots", {})
    
    lines = [
        "# HEGEMON Human-in-the-Loop Summary",
        "",
        f"**Intervention Mode:** {state['intervention_mode']}",
        f"**Total Feedback Submissions:** {len(feedback_history)}",
        f"**Checkpoints Reached:** {len(checkpoints_reached)}",
        "",
        "---",
        "",
    ]
    
    if feedback_history:
        lines.extend([
            "## 📝 Feedback History",
            "",
        ])
        
        for i, fb in enumerate(feedback_history, 1):
            lines.extend([
                f"### Feedback #{i}",
                f"- **Checkpoint:** {fb.get('checkpoint', 'unknown')}",
                f"- **Decision:** {fb.get('decision', 'unknown')}",
                f"- **Timestamp:** {fb.get('timestamp', 'unknown')}",
                "",
            ])
            
            if fb.get('guidance'):
                lines.extend([
                    f"**Guidance:**",
                    f"> {fb['guidance']}",
                    "",
                ])
            
            if fb.get('priority_claims'):
                lines.append("**Priority Claims:**")
                for claim in fb['priority_claims']:
                    lines.append(f"- {claim}")
                lines.append("")
            
            if fb.get('flagged_concerns'):
                lines.append("**Flagged Concerns:**")
                for concern in fb['flagged_concerns']:
                    lines.append(f"- {concern}")
                lines.append("")
            
            lines.append("---")
            lines.append("")
    else:
        lines.extend([
            "## No Human Feedback",
            "",
            "Debate ran in **observer mode** or no interventions were made.",
            "",
        ])
    
    if revision_counts:
        lines.extend([
            "## 🔄 Revision Statistics",
            "",
        ])
        
        for checkpoint, count in revision_counts.items():
            lines.append(f"- **{checkpoint}:** {count} revision(s)")
        
        lines.append("")
    
    content = "\n".join(lines)
    
    Path(output_path).write_text(content, encoding="utf-8")
    
    return output_path

# Export HITL summary
hitl_report = export_hitl_feedback_summary(
    final_state,
    output_path="hitl_summary.md"
)

print(f"✅ Exported {len(exported['json'])} epistemic profiles")
print(f"✅ Layer 2 Report: {report}")
print(f"✅ HITL Summary: {hitl_report}")

print("\n" + "=" * 80)
print(f"✅ Debate completed in {final_state['cycle_count']} cycles")
print(f"   Final consensus: {final_state.get('current_consensus_score', 0.0):.2f}")
print(f"   Total contributions: {len(final_state['contributions'])}")
print(f"   Human interventions: {len(final_state['human_feedback_history'])}")
print("=" * 80)


print(f"✅ Exported {len(exported['json'])} profiles")
print(f"✅ Report: {report}")






# ============================================================================
# BONUS: HITL-specific metrics (NEW!)
# ============================================================================

print(f"   Intervention mode: {final_state['intervention_mode']}")
print(f"   Human feedback count: {len(final_state['human_feedback_history'])}")
print(f"   Checkpoints reached: {len(final_state['checkpoint_snapshots'])}")
print("=" * 80)

2025-10-03 13:51:31,874 - hegemon.agents - INFO - 🔥 KATALIZATOR (anthropic/claude-sonnet-4-5-20250929): Generating Thesis (Cycle 1)
2025-10-03 13:51:31,884 - hegemon.agents - INFO - ✅ KATALIZATOR: Thesis generated (1836 chars)
2025-10-03 13:51:31,887 - hegemon.explainability.concepts - INFO - ✅ Loaded 100 concepts from /home/jupyter/olga_zydziak/version_beta/Folder/hegemon_mvp/hegemon/explainability/data/concepts.json
2025-10-03 13:51:31,888 - hegemon.explainability.classifier - INFO - ✅ ConceptClassifier initialized: Vertex AI gemini-2.0-flash (project=dark-data-discovery, location=us-central1), cache_size=1000
2025-10-03 13:51:31,889 - hegemon.explainability.epistemic - INFO - ClaimExtractor initialized: Vertex AI gemini-2.0-flash-exp (project=dark-data-discovery, location=us-central1)
2025-10-03 13:51:31,890 - hegemon.agents - INFO - Layer 2 (Epistemic Uncertainty) enabled
2025-10-03 13:51:31,891 - hegemon.explainability.collector - INFO - ✅ ExplainabilityCollector initialized
2025-

🎭 Starting dialectical debate...
⏳ This will take 2-5 minutes...



2025-10-03 13:51:32,077 - hegemon.agents - INFO - ✅ KATALIZATOR: Thesis generated (2134 chars)
2025-10-03 13:51:32,078 - hegemon.explainability.collector - INFO - 🔍 Collecting explainability for Katalizator (Cycle 5)
2025-10-03 13:51:32,081 - hegemon.explainability.classifier - INFO - ✅ Classification successful: 2ms, top_concept=('data_driven_approach', 0.7)
2025-10-03 13:51:32,084 - hegemon.explainability.epistemic - INFO - Claim extraction successful: 14 claims, 1ms, avg_conf=0.63
2025-10-03 13:51:32,084 - hegemon.explainability.collector - INFO - ✅ Explainability collected for Katalizator: L6(2ms), L2(1ms, 14 claims)
2025-10-03 13:51:32,086 - hegemon.hitl.checkpoints - INFO - 🔭 Observer mode active - skipping post_thesis checkpoint
2025-10-03 13:51:32,087 - hegemon.agents - INFO - ⚔️ SCEPTYK (google/gemini-2.5-pro via Vertex AI): Generating Antithesis (Cycle 5)
2025-10-03 13:51:32,088 - langchain_google_vertexai.chat_models - WARNING - Unexpected argument 'timeout' provided to Chat


EXPORTING LAYER 2 DATA...
✅ Exported 16 epistemic profiles
✅ Layer 2 Report: epistemic_comparison.md
✅ HITL Summary: hitl_summary.md

✅ Debate completed in 5 cycles
   Final consensus: 0.62
   Total contributions: 16
   Human interventions: 0
✅ Exported 16 profiles
✅ Report: epistemic_comparison.md
   Intervention mode: observer
   Human feedback count: 0
   Checkpoints reached: 0


In [7]:
# Komórka 6: Inspect contributions
print("📋 DEBATE CONTRIBUTIONS:\n")

for i, contrib in enumerate(final_state["contributions"], 1):
    print(f"{i}. {contrib.agent_id} (Cycle {contrib.cycle}) - {contrib.type}")
    print(f"   Content length: {len(contrib.content)} chars")
    
    # Check if has explainability
    if contrib.explainability and contrib.explainability.semantic_fingerprint:
        vector = contrib.explainability.semantic_fingerprint
        print(f"   ✅ Explainability: {vector.processing_time_ms}ms, Cache: {vector.cache_hit}")
        
        # Show top 3 concepts
        top_3 = vector.top_k(3)
        print(f"   Top concepts: {', '.join([f'{cid}({score:.2f})' for cid, score in top_3])}")
    else:
        print(f"   ❌ No explainability data")
    
    print()

📋 DEBATE CONTRIBUTIONS:

1. Katalizator (Cycle 1) - Thesis
   Content length: 1836 chars
   ✅ Explainability: 3ms, Cache: False
   Top concepts: urgency(0.60), innovation_bias(0.60), efficiency_focus(0.60)

2. Sceptyk (Cycle 1) - Antithesis
   Content length: 289 chars
   ✅ Explainability: 2ms, Cache: False
   Top concepts: skepticism(0.70), risk_aversion(0.65), downside_protection(0.60)

3. Gubernator (Cycle 1) - Evaluation
   Content length: 1395 chars
   ✅ Explainability: 2ms, Cache: False
   Top concepts: innovation_bias(0.50), analytical_thinking(0.50), pragmatism(0.50)

4. Katalizator (Cycle 2) - Thesis
   Content length: 2017 chars
   ✅ Explainability: 3ms, Cache: False
   Top concepts: risk_reward_balance(0.70), iterative_pacing(0.70), incremental_improvement(0.70)

5. Sceptyk (Cycle 2) - Antithesis
   Content length: 533 chars
   ✅ Explainability: 2ms, Cache: False
   Top concepts: aggressive_timeline(0.70), pessimism(0.60), precautionary_principle(0.50)

6. Gubernator (Cycle 

In [8]:
# Komórka 7: Visualize semantic fingerprints
visualizer = HeatmapGenerator()

print("📊 SEMANTIC FINGERPRINTS\n")

for contrib in final_state["contributions"]:
    if contrib.explainability and contrib.explainability.semantic_fingerprint:
        vector = contrib.explainability.semantic_fingerprint
        
        print("=" * 80)
        print(f"{contrib.agent_id} - Cycle {contrib.cycle} ({contrib.type})")
        print("=" * 80)
        
        heatmap = visualizer.generate_text_heatmap(vector, top_k=10)
        print(heatmap)
        print("\n")

📊 SEMANTIC FINGERPRINTS

Katalizator - Cycle 1 (Thesis)
SEMANTIC FINGERPRINT

Top 10 Concepts:

 1. Urgency                        0.60 ████████████░░░░░░░░
 2. Innovation Bias                0.60 ████████████░░░░░░░░
 3. Efficiency Focus               0.60 ████████████░░░░░░░░
 4. Data-Driven Approach           0.60 ████████████░░░░░░░░
 5. Internal Efficiency Focus      0.60 ████████████░░░░░░░░
 6. Optimism                       0.60 ████████████░░░░░░░░
 7. Systems Thinking               0.60 ████████████░░░░░░░░
 8. Bold Experimentation           0.50 ██████████░░░░░░░░░░
 9. Upside Seeking                 0.50 ██████████░░░░░░░░░░
10. Aggressive Timeline            0.50 ██████████░░░░░░░░░░

----------------------------------------------------------------------
By Category (Top 3 per category):

## Risk Orientation
   Bold Experimentation           0.50 █████
   Upside Seeking                 0.50 █████
   Risk Tolerance                 0.40 ████

## Time Orientation
   Urgency  

In [9]:
# Komórka 8: Compare Katalizator vs Sceptyk
kataliz_contribs = [c for c in final_state["contributions"] if c.agent_id == "Katalizator"]
sceptyk_contribs = [c for c in final_state["contributions"] if c.agent_id == "Sceptyk"]

if kataliz_contribs and sceptyk_contribs:
    kataliz_last = kataliz_contribs[-1]
    sceptyk_last = sceptyk_contribs[-1]
    
    if (kataliz_last.explainability and sceptyk_last.explainability and
        kataliz_last.explainability.semantic_fingerprint and 
        sceptyk_last.explainability.semantic_fingerprint):
        
        print("🔄 COMPARISON: Katalizator vs Sceptyk\n")
        
        comparison = visualizer.generate_comparison_text(
            kataliz_last.explainability.semantic_fingerprint,
            sceptyk_last.explainability.semantic_fingerprint,
            label1="Katalizator",
            label2="Sceptyk",
            top_k=10
        )
        print(comparison)
        
        # Similarity score
        similarity = kataliz_last.explainability.semantic_fingerprint.compare(
            sceptyk_last.explainability.semantic_fingerprint
        )
        print(f"\n📊 Cognitive Similarity: {similarity:.3f}")
        print(f"   Interpretation: {'Similar thinking' if similarity > 0.7 else 'Divergent thinking' if similarity < 0.3 else 'Moderately different'}")

🔄 COMPARISON: Katalizator vs Sceptyk

COMPARISON: Katalizator vs Sceptyk

Cosine Similarity: 0.914

Concept                            Katalizator         Sceptyk        Δ
--------------------------------------------------------------------------------
Upside Seeking                   0.00              1.00 ██████████   +1.00
Caution                          0.00              0.80 ████████     +0.80
Aggressive Timeline              0.60 ██████       0.80 ████████     +0.20
Systematic Methodology           0.70 ███████      0.00              -0.70
Pessimism                        0.00              0.70 ███████      +0.70
Structured Presentation          0.70 ███████      0.00              -0.70
Pragmatism                       0.70 ███████      0.00              -0.70
Urgency                          0.00              0.70 ███████      +0.70
Interdependency Awareness        0.70 ███████      0.70 ███████      +0.00
Data-Driven Approach             0.70 ███████      0.00              -0.

In [10]:
# Komórka 9: Save results
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Save JSON
output_file = output_dir / "hegemon_jupyter_output.json"

output_data = {
    "mission": final_state["mission"],
    "total_cycles": final_state["cycle_count"],
    "final_consensus_score": final_state.get("current_consensus_score", 0.0),
    "contributions": [c.model_dump() for c in final_state["contributions"]],
    "final_plan": final_state["final_plan"].model_dump() if final_state["final_plan"] else None,
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, default=str)

print(f"✅ Output saved to: {output_file}")

# Save heatmaps
for contrib in final_state["contributions"]:
    if contrib.explainability and contrib.explainability.semantic_fingerprint:
        vector = contrib.explainability.semantic_fingerprint
        heatmap = visualizer.generate_text_heatmap(vector, top_k=20)
        
        heatmap_file = output_dir / f"heatmap_{contrib.agent_id}_cycle{contrib.cycle}.txt"
        with open(heatmap_file, "w", encoding="utf-8") as f:
            f.write(heatmap)
        
        print(f"✅ Heatmap saved: {heatmap_file.name}")

print("\n🎉 All results saved!")

✅ Output saved to: output/hegemon_jupyter_output.json
✅ Heatmap saved: heatmap_Katalizator_cycle1.txt
✅ Heatmap saved: heatmap_Sceptyk_cycle1.txt
✅ Heatmap saved: heatmap_Gubernator_cycle1.txt
✅ Heatmap saved: heatmap_Katalizator_cycle2.txt
✅ Heatmap saved: heatmap_Sceptyk_cycle2.txt
✅ Heatmap saved: heatmap_Gubernator_cycle2.txt
✅ Heatmap saved: heatmap_Katalizator_cycle3.txt
✅ Heatmap saved: heatmap_Sceptyk_cycle3.txt
✅ Heatmap saved: heatmap_Gubernator_cycle3.txt
✅ Heatmap saved: heatmap_Katalizator_cycle4.txt
✅ Heatmap saved: heatmap_Sceptyk_cycle4.txt
✅ Heatmap saved: heatmap_Gubernator_cycle4.txt
✅ Heatmap saved: heatmap_Katalizator_cycle5.txt
✅ Heatmap saved: heatmap_Sceptyk_cycle5.txt
✅ Heatmap saved: heatmap_Gubernator_cycle5.txt
✅ Heatmap saved: heatmap_Syntezator_cycle5.txt

🎉 All results saved!


In [11]:
# ═══════════════════════════════════════════════════════════════════════════
# KOMÓRKA: Export Layer 2 (Epistemic Uncertainty) Data
# ═══════════════════════════════════════════════════════════════════════════

from hegemon.explainability.visualizer import (
    export_all_epistemic_profiles,
    create_epistemic_comparison_report,
)

# 1. Export individual profiles (JSON + Text)
exported_files = export_all_epistemic_profiles(
    contributions=final_state["contributions"],
    output_dir="epistemic_exports",
    format="both",  # "json", "text", or "both"
)

print("=" * 70)
print("EPISTEMIC PROFILES EXPORTED")
print("=" * 70)
print(f"JSON files: {len(exported_files['json'])}")
print(f"Text files: {len(exported_files['text'])}")
print()

# Show example
if exported_files['text']:
    print(f"Example text file: {exported_files['text'][0]}")
    with open(exported_files['text'][0], 'r') as f:
        print(f.read()[:500])
        print("...")

print()

# 2. Create comparison report (Markdown)
report_path = create_epistemic_comparison_report(
    contributions=final_state["contributions"],
    output_path="epistemic_comparison.md",
)

print(f"Comparison report: {report_path}")
print()

# 3. Show summary
print("=" * 70)
print("LAYER 2 SUMMARY")
print("=" * 70)

for contrib in final_state["contributions"]:
    if contrib.explainability and contrib.explainability.epistemic_profile:
        profile = contrib.explainability.epistemic_profile
        stats = profile.get_summary_stats()
        
        print(f"{contrib.agent_id} (Cycle {contrib.cycle}):")
        print(f"  Claims: {stats['total_claims']}")
        print(f"  Avg Confidence: {stats['aggregate_confidence']:.2f}")
        print(f"  High/Med/Low: {stats['high_confidence_count']}/"
              f"{stats['medium_confidence_count']}/{stats['low_confidence_count']}")
        
        # Show lowest confidence claim
        low_claims = profile.get_low_confidence_claims(threshold=0.5)
        if low_claims:
            lowest = min(low_claims, key=lambda c: c.confidence)
            print(f"  ⚠️ Lowest: [{lowest.confidence:.2f}] {lowest.claim_text[:80]}...")
        
        print()

2025-10-03 13:52:32,905 - hegemon.explainability.visualizer - INFO - Epistemic profile exported to epistemic_exports/epistemic_Katalizator_cycle1.json
2025-10-03 13:52:32,908 - hegemon.explainability.visualizer - INFO - Epistemic profile exported to epistemic_exports/epistemic_Katalizator_cycle1.txt
2025-10-03 13:52:32,909 - hegemon.explainability.visualizer - INFO - Epistemic profile exported to epistemic_exports/epistemic_Sceptyk_cycle1.json
2025-10-03 13:52:32,910 - hegemon.explainability.visualizer - INFO - Epistemic profile exported to epistemic_exports/epistemic_Sceptyk_cycle1.txt
2025-10-03 13:52:32,912 - hegemon.explainability.visualizer - INFO - Epistemic profile exported to epistemic_exports/epistemic_Gubernator_cycle1.json
2025-10-03 13:52:32,913 - hegemon.explainability.visualizer - INFO - Epistemic profile exported to epistemic_exports/epistemic_Gubernator_cycle1.txt
2025-10-03 13:52:32,915 - hegemon.explainability.visualizer - INFO - Epistemic profile exported to epistemi

EPISTEMIC PROFILES EXPORTED
JSON files: 16
Text files: 16

Example text file: epistemic_exports/epistemic_Katalizator_cycle1.txt
EPISTEMIC UNCERTAINTY PROFILE

Agent: Katalizator
Cycle: 1
Model: gemini-2.0-flash-exp (Vertex AI)
Processing time: 2ms

----------------------------------------------------------------------
SUMMARY STATISTICS
----------------------------------------------------------------------

Total claims: 14
Aggregate confidence: 0.56

Confidence distribution:
  Hig
...

Comparison report: epistemic_comparison.md

LAYER 2 SUMMARY
Katalizator (Cycle 1):
  Claims: 14
  Avg Confidence: 0.56
  High/Med/Low: 5/6/3
  ⚠️ Lowest: [0.30] Within 12 months, a 3-4x increase in the number of experiments is expected due t...

Sceptyk (Cycle 1):
  Claims: 3
  Avg Confidence: 0.57
  High/Med/Low: 1/1/1
  ⚠️ Lowest: [0.40] The proposal drastically underestimates initial......

Gubernator (Cycle 1):
  Claims: 10
  Avg Confidence: 0.82
  High/Med/Low: 9/1/0

Katalizator (Cycle 2):
  Clai

In [12]:
# Komórka 10: Display final plan
if final_state["final_plan"]:
    plan = final_state["final_plan"]
    
    print("=" * 80)
    print("📋 FINAL STRATEGIC PLAN")
    print("=" * 80)
    print()
    
    print("## Mission Overview")
    print(plan.mission_overview)
    print()
    
    print(f"## Required Agents ({len(plan.required_agents)})")
    for i, agent in enumerate(plan.required_agents, 1):
        print(f"\n{i}. {agent.role}")
        print(f"   {agent.description}")
        print(f"   Skills: {', '.join(agent.required_skills[:3])}...")
    print()
    
    print(f"## Workflow ({len(plan.workflow)} steps)")
    for step in plan.workflow[:5]:  # Show first 5
        deps = f" (depends on: {step.dependencies})" if step.dependencies else ""
        print(f"{step.step_id}. {step.description}{deps}")
    if len(plan.workflow) > 5:
        print(f"   ... and {len(plan.workflow) - 5} more steps")
    print()
    
    print("## Risk Analysis")
    print(plan.risk_analysis)
    print()
    print("=" * 80)
else:
    print("❌ No final plan generated")

📋 FINAL STRATEGIC PLAN

## Mission Overview
Design and implement production-ready ML pipeline balancing architectural sophistication with pragmatic delivery timelines. Phase 1 (Q1): Deploy single high-ROI model with minimal infrastructure (Airflow, MLflow, FastAPI). Phase 2 (Q2-Q3): Extract reusable patterns, introduce selective feature store for proven reuse cases. Phase 3 (Q4): Establish bounded contexts architecture with explicit contracts only after validating 5+ models in production. Expected: 70% reduction in deployment time, 15+ models by year-end, avoiding premature complexity while maintaining evolution capacity.

## Required Agents (5)

1. ML Platform Architect
   Designs overall pipeline architecture, defines bounded contexts and API contracts, ensures scalability without overengineering
   Skills: MLflow, Kubeflow/Airflow orchestration, API design patterns...

2. Data Engineering Lead
   Builds data ingestion pipelines, implements schema validation with Great Expectations, 

In [ ]:
# Komórka 1: Interactive config
import ipywidgets as widgets
from IPython.display import display
import os

# Create toggles
enable_expl = widgets.Checkbox(
    value=True,
    description='Enable Explainability',
    style={'description_width': 'initial'}
)

enable_fingerprint = widgets.Checkbox(
    value=True,
    description='Semantic Fingerprint',
    style={'description_width': 'initial'}
)

model_dropdown = widgets.Dropdown(
    options=['gemini-2.0-flash-exp', 'gemini-1.5-pro', 'gpt-4o-mini'],
    value='gemini-2.0-flash-exp',
    description='Classifier Model:',
    style={'description_width': 'initial'}
)

def apply_config(btn):
    os.environ["HEGEMON_EXPLAINABILITY_ENABLED"] = str(enable_expl.value).lower()
    os.environ["HEGEMON_EXPLAINABILITY_SEMANTIC_FINGERPRINT"] = str(enable_fingerprint.value).lower()
    os.environ["HEGEMON_EXPLAINABILITY_CLASSIFIER_MODEL"] = model_dropdown.value
    print("✅ Configuration applied!")

apply_btn = widgets.Button(description="Apply Config", button_style='success')
apply_btn.on_click(apply_config)

# Display
display(enable_expl, enable_fingerprint, model_dropdown, apply_btn)

In [ ]:
# W Jupyter - restart kernel i test
from hegemon.explainability.collector import ExplainabilityCollector

# Sprawdź czy ma wszystkie metody
collector_methods = [m for m in dir(ExplainabilityCollector) if not m.startswith('_')]
print("Public methods:", collector_methods)
# Powinno pokazać: ['collect', 'get_stats']

private_methods = [m for m in dir(ExplainabilityCollector) if m.startswith('_collect')]
print("Private collection methods:", private_methods)
# Powinno pokazać: ['_collect_epistemic_profile', '_collect_semantic_fingerprint']